In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
import dask
from datetime import timedelta
import glob
import itertools
%matplotlib inline

# plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import textwrap

import matplotlib as mpl
mpl.use('agg')

# # import personal modules
# sys.path.append('../modules')
# import ar_funcs

dask.config.set(**{'array.slicing.split_large_chunks': True})

sys.path.append('../../mclimate_tool_cw3e')
from build_html_table import create_html_table
import custom_cmaps as ccmap
from plotter import draw_basemap, plot_mclimate_forecast
# from plotter import draw_basemap

import mclimate_funcs as mclim_func

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
### list of dates currently ran
## '20051120' 'GEFSv12_reforecast' '20051124'
## '20070607' 'GEFSv12_reforecast'
## '20150813' 'GEFSv12_reforecast' '20150818'
## '20201128' 'GEFS_archive' '20201202'
## '20231117' 'GEFS_archive''20231120'
## '20231114' 'GEFS_archive'

In [4]:
%%time
######################
### VARS TO UPDATE ###
######################
fdate = '20201128' ## initialization date in YYYYMMDD format
fdate2 = '20201202' ## date of impact
model = 'GEFS_archive' ## 'GEFSv12_reforecast', 'GFS', 'GEFS', 'GEFS_archive'
map_ext = [-170., -120., 40., 65.] ## map extent [minlon, maxlon, minlat, maxlat]
# [-170., -125., 50., 75.] AK
# [-170., -120., 50., 75.] SEAK/NPAC
table_ext = [-141., -130., 54.5, 60.] ## extent to choose the maximum value from for the table [minlon, maxlon, minlat, maxlat]
# [-141., -130., 54.5, 60.] SEAK
# [-160., -145., 65., 70.] Northern AK
fig_path = path_to_figs + 'images_historical/mclimate_{0}/'.format(fdate)
os.makedirs(os.path.dirname(fig_path), exist_ok=True)

####################################
### COMPARE FORECAST TO MCLIMATE ###
####################################
var_lst = ['ivt', 'freezing_level', 'uv1000']
ds_lst = []
ds_lst2 = []
fc_lst = []
for i, varname in enumerate(var_lst):
    forecast, ds = mclim_func.run_compare_mclimate_forecast(varname, fdate, model, server='expanse')
    fc_lst.append(forecast)
    ds_lst.append(ds)

    if varname == 'uv1000':
            ds = ds.rename({'mclimate': 'uv'})
    else:
        ds = ds.rename({'mclimate': varname})
    ds_lst2.append(ds)
    
### merge the datasets
ds3 = xr.merge(ds_lst2)
ds3 = ds3.sortby('lat')

ds3

11 28
11 28
11 28
CPU times: user 3.52 s, sys: 1.25 s, total: 4.77 s
Wall time: 19.6 s


<xarray.Dataset>
Dimensions:         (step: 40, lat: 121, lon: 140)
Coordinates:
  * lat             (lat) float64 10.0 10.5 11.0 11.5 ... 68.5 69.0 69.5 70.0
  * lon             (lon) float64 -179.5 -179.0 -178.5 ... -111.0 -110.5 -110.0
  * step            (step) int64 6 12 18 24 30 36 42 ... 210 216 222 228 234 240
    init_date       datetime64[ns] 2020-11-28
    surface         float64 0.0
    isothermZero    float64 0.0
    isobaricInhPa   float64 1e+03
Data variables:
    ivt             (step, lat, lon) float64 0.9 0.91 0.9 0.9 ... nan nan nan
    freezing_level  (step, lat, lon) float64 nan nan nan nan ... nan nan nan nan
    uv              (step, lat, lon) float64 0.93 0.95 0.94 0.94 ... nan nan nan

In [5]:
# ####################
# ### CREATE PLOTS ###
# ####################
# region_lst = ['NPAC']
# step_lst = ds3.step.values
# print('...Writing plots')
# for i, (varname, ds, fc) in enumerate(zip(var_lst, ds_lst, fc_lst)):
#     for region in region_lst:
#         for step in step_lst:
#             print('...for {0}, {1}, {2}'.format(varname, region, step))
#             out_fname = fig_path + '{2}_{0}_mclimate_F{1}'.format(varname, step, region)
#             ## write plots
#             plot_mclimate_forecast(ds, fc, step=step, varname=varname, fname=out_fname, ext_name=region, historical=True)

# step_lst = ds.step.values
# for i, step in enumerate(step_lst):
#     print(step)
#     out_fname = fig_path + 'SEAK_mclimate_F{0}'.format(step)
#     plot_mclimate_forecast_four_panel(ds, fc, step, out_fname, domain="SEAK")
#     out_fname = fig_path + 'NPAC_mclimate_F{0}'.format(step)
#     plot_mclimate_forecast_four_panel(ds, fc, step, out_fname, domain="NPAC")

In [6]:
###################
### BUILD TABLE ###
###################
df = create_html_table(ds3, domain="SEAK")
df_html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)
df

In [7]:
# ## write to text file
# html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# # write html to file
# text_file = open("../out/table_{0}.html".format(fdate), "w")
# text_file.write(html)
# text_file.close()

In [8]:
#######################
### WRITE HTML FILE ###
#######################
print('...Writing HTML file')
out_fname = "../out/mclimate_tool_historical_{0}.html".format(fdate2)

with open('../../mclimate_tool_cw3e/out/html_text.txt', mode='r') as in_file, \
     open('../../mclimate_tool_cw3e/out/html_text2_{0}.txt'.format(fdate2), mode='r') as in_file2, \
     open(out_fname, mode='w') as out_file:

    # A file is iterable
    # We can read each line with a simple for loop
    for line in in_file:
        out_file.write(line)
        
        
    ## now add in the table
    out_file.write(df_html)

    ## now add the last few lines
    for line in in_file2:
        out_file.write(line)
        
    out_file.close()

...Writing HTML file
